# Import packages

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Create spark session and load data

In [21]:
spark = SparkSession.builder.appName("analyze").getOrCreate()

file_path = "/home/dotronghiep/Documents/Datasets/Bank_Credit_Default/loan/loan.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [22]:
df.show()

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------

# Take useful sample with direct label

In [32]:
# check labels
df.select('loan_status').distinct().show()

+--------------------+
|         loan_status|
+--------------------+
|          Fully Paid|
|     In Grace Period|
|         Charged Off|
|  Late (31-120 days)|
|             Current|
|   Late (16-30 days)|
|             Default|
|            Oct-2015|
|Does not meet the...|
|Does not meet the...|
+--------------------+



In [ ]:
# just take 3 labels: 'Fully Paid' is 1, 'Charged Off' and 'Default' are 0



# Dataset's basic infors

In [23]:
num_rows = df.count()
num_columns = len(df.columns)
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 2260668
Number of columns: 145


In [24]:
df.columns

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [25]:
# count the number of missing values in each column
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+-------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+-----------+----------+-------+-------+-------+-----+--------+----------+----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--

# Delete columns have more than 10000 null values

In [26]:
# convert the row to dictionary
null_counts_row = null_counts.collect()[0].asDict()

# find the list have number of null values greater than 10000
columns_to_drop = [k for k, v in null_counts_row.items() if v > 10000]

# drop these columns
df_cleaned = df.drop(*columns_to_drop)

In [27]:
len(df_cleaned.columns)

51

In [28]:
# count the number of missing values in each column of new dataframe
null_counts_cleaned = df_cleaned.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_cleaned.columns])
null_counts_cleaned.show()

+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+-------+-----------+----------+-------+--------+----------+----+-----------+----------------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------------+--------------------------+-----------+----------------+--------------+------------------------+-----------+--------------------+---------+-------------+-------------------+--------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|issue_d|loan_status|pymnt_plan|purpose|zip_code|addr_state| dti|delinq_2yrs|earliest_cr_line|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_

In [29]:
df_cleaned.show()

+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+--------+-----------+----------+------------------+--------+----------+-----+-----------+----------------+--------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------------+--------------------------+-----------+----------------+--------------+------------------------+-----------+--------------------+---------+-------------+-------------------+--------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|pymnt_plan|           purpose|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|inq_last_6mths|open_acc|pub_rec|revol_

In [30]:
df_cleaned.select('issue_d').distinct().show()

+--------+
| issue_d|
+--------+
|Sep-2018|
|Nov-2018|
|Dec-2018|
|Aug-2018|
|Oct-2018|
|Jul-2018|
|Apr-2018|
|Jun-2018|
|May-2018|
|Mar-2018|
|Jul-2016|
|Aug-2016|
|Jan-2018|
|Sep-2016|
|Feb-2018|
|May-2016|
|Apr-2016|
|Mar-2016|
|Feb-2016|
|Jun-2016|
+--------+
only showing top 20 rows



+--------------------+
|         loan_status|
+--------------------+
|          Fully Paid|
|     In Grace Period|
|         Charged Off|
|  Late (31-120 days)|
|             Current|
|   Late (16-30 days)|
|             Default|
|            Oct-2015|
|Does not meet the...|
|Does not meet the...|
+--------------------+

